In [1]:
import pandas as pd
import os
import utility
import numpy as np

In [2]:
#import data: 
datapath = os.getcwd() + '/Data/'
holidays = pd.read_csv(datapath + 'holidays_events.csv')

In [3]:
# Check how holidays are distributed every year. 
holidays['date'] = pd.to_datetime(holidays['date'])
holidays['year'] = holidays.date.dt.year
holidays['month-day'] = holidays['date'].dt.strftime('%m-%d')

# Show all rows
pd.set_option('display.max_rows', None)
# First: National
national = holidays[(holidays.locale == 'National') & (holidays.type == 'Holiday')]

## Type of holidays 
The different types of holiday indicates for a particular store in a particlular day, it can be:

- holiday, celebrated
- holiday, not celebrated
- not holiday, celebrated 
- not holiday, not celebrated
- event


We don't know if for a particular holiday, whether it's a day off or not is going to affect the sale. So I'll also add "whether it's celebrated (day off)" as a feature. 

## location and description 
There're three locations. Based on the location, we can determine wheter a particular store celebrate the holiday. 

Different holidays will likely have different shopping culture. Holidays should be considered separately. 

So at first glance, I will add three columns:
- Local holiday (0 is none, 1 - 27 is the different discription of the holidays.)
- Regional holiday (0 is None, 1 - 4 is four different holidays.)
- National holiday (0 is None, 1 - 29 are different holidays.)
- National Events (0 is None, 1 - 43 are different events.) 
  - Event is not recruiting. Hence it should be labeled separately. 

One problem, 2012-12-31	is both Puente Primer dia del ano	(Bridge) and Primer dia del ano-1	(Additional)
It termed out Puente Primer dia del ano	means "The bridge for new year" , Primer dia del ano means "new year eve".

Other than that, there's no overlapping holidays. 

Notice that "Bridge", "Work Day" and "Transfer" Changes every year based on the day of the week. 
While "Additional" only goes with the holiday it self. 

# Different holidays 

Not all holiday is repeated in the same day every year. 

In [4]:

table_national = national.pivot_table(index='month-day', columns='year', values='description', aggfunc=lambda x: ', '.join(x)).fillna(0)
# print(table_national)

In [5]:
# Then Regional 
Regional = holidays[(holidays.locale == 'Regional') & (holidays.type == 'Holiday')]
table_regional = Regional.pivot_table(index='month-day', columns='year', values='description', aggfunc=lambda x: ', '.join(x)).fillna(0)
# print(table_regional)

In [6]:
# Finally, local: 

Local = holidays[(holidays.locale == 'Local') & (holidays.type == 'Holiday')]
table_local = Local.pivot_table(index='month-day', columns='year', values='description', aggfunc=lambda x: ', '.join(x)).fillna(0)
# print(table_local)

Not All national holidays are of the same date. \
Regional holidays are of the day date. 

Local holiday is very wild. 
The  Fundacion de Guayaquil is celebrated in different ways every time. 

# Summary on holiday types 

Because how complicated the different holidays are, it's easier to use the holiday.csv as a table and not try to predict the holiday dates. 

Treat every discription separetly. 
To deal with 2012-12-31, label it as "Primer dia del ano" for now. Since new year eve affects more than just a "bridge day". (If any thing goes wrong, this is the "error" we introduce. )



In [7]:


# Create a new dataset: 
Local_holidays = holidays[holidays.locale == 'Local']
Local_holidays['local_holiday'] = Local_holidays['description']
# Set 'Local_celebrated' to True where 'type' is 'Holiday'
Local_holidays['Local_celebrated'] = Local_holidays['type'].isin(['Holiday','Bridge', 'Additional','Transfer'])
# Update 'Local_celebrated' to False where 'transferred' is True
Local_holidays.loc[Local_holidays['transferred'] == True, 'Local_celebrated'] = False
# Take only the needed 
Local_holidays = Local_holidays[['date', 'local_holiday', 'locale_name','Local_celebrated']]

# For regional too 
Regional_holidays = holidays[holidays.locale == 'Regional']
Regional_holidays['Regional_holiday'] = Regional_holidays['description']
# Set 'Local_celebrated' to True where 'type' is 'Holiday'
Regional_holidays['Regional_celebrated'] = Regional_holidays['type'].isin(['Holiday','Bridge', 'Additional','Transfer'])
# Update 'Local_celebrated' to False where 'transferred' is True
Regional_holidays.loc[Regional_holidays['transferred'] == True, 'Regional_celebrated'] = False
# Take only the needed 
Regional_holidays = Regional_holidays[['date', 'Regional_holiday', 'locale_name','Regional_celebrated']]

# Same with National 
National_holidays = holidays[holidays.locale == 'National']
National_holidays['National_holiday'] = National_holidays['description']
# Set 'Local_celebrated' to True where 'type' is 'Holiday'
National_holidays['National_celebrated'] = National_holidays['type'].isin(['Holiday','Bridge', 'Additional','Transfer'])
# Update 'Local_celebrated' to False where 'transferred' is True
National_holidays.loc[National_holidays['transferred'] == True, 'National_celebrated'] = False
# Take only the needed 
National_holidays = National_holidays[['date', 'National_holiday', 'locale_name','National_celebrated']]


/var/folders/3d/wg74p6ds4615xyxvs58d4f_40000gn/T/ipykernel_56338/1476164347.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Local_holidays['local_holiday'] = Local_holidays['description']
/var/folders/3d/wg74p6ds4615xyxvs58d4f_40000gn/T/ipykernel_56338/1476164347.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Local_holidays['Local_celebrated'] = Local_holidays['type'].isin(['Holiday','Bridge', 'Additional','Transfer'])
/var/folders/3d/wg74p6ds4615xyxvs58d4f_40000gn/T/ipykernel_56338/1476164347.py:1

# Study the effect of holidays 


In [8]:
# Import the combined training file with the correct holiday merge. 
#import data: 
# datapath = os.getcwd() + '/Data/'
#df = pd.read_csv(datapath + 'combined-2.csv')
df = pd.read_csv(datapath + 'merged_train.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/fdy233/Library/CloudStorage/OneDrive-RiceUniversity/Documents/RUNCODE/Untitled/Erdos_Grocery_Sales/Data/merged_train.csv'

In [ ]:
# Uncomment this to see a summary of the df. 
#utility.summary(df)
#utility.df_info(df)

In [ ]:
df_hol = df[(df.hol_Nat == 1) | (df.hol_Reg == 1) | (df.hol_Loc == 1)  ]
df_hol = df_hol[['date', 'year','month','day','day_of_week','city','state','hol_Nat','hol_Nat_name',
                 'hol_Reg','hol_Reg_name','hol_Loc','hol_loc_name','transferred']]
df_hol = df_hol.drop_duplicates() 
# Check if there's holiday in Aug after 15: 
print(df_hol[(df_hol.month == 8) & (df_hol.day > 15)])

               date  year  month  day  day_of_week    city       state  \
358512   2013-08-24  2013      8   24            5  Ambato  Tungurahua   
920370   2014-08-24  2014      8   24            6  Ambato  Tungurahua   
1505526  2015-08-24  2015      8   24            0  Ambato  Tungurahua   
2136618  2016-08-24  2016      8   24            2  Ambato  Tungurahua   

         hol_Nat hol_Nat_name  hol_Reg hol_Reg_name  hol_Loc  \
358512       0.0          NaN      0.0          NaN      1.0   
920370       0.0          NaN      0.0          NaN      1.0   
1505526      0.0          NaN      0.0          NaN      1.0   
2136618      0.0          NaN      0.0          NaN      1.0   

                hol_loc_name transferred  
358512   Fundacion de Ambato           1  
920370   Fundacion de Ambato         1.0  
1505526  Fundacion de Ambato           1  
2136618  Fundacion de Ambato         1.0  


In [ ]:
#Confirm with the orginal holiday csv:
holidays[holidays['month-day'] == '08-24']

,date,type,locale,locale_name,description,transferred,year,month-day
16,2012-08-24,Holiday,Local,Ambato,Fundacion de Ambato,False,2012,08-24
69,2013-08-24,Holiday,Local,Ambato,Fundacion de Ambato,False,2013,08-24
132,2014-08-24,Holiday,Local,Ambato,Fundacion de Ambato,False,2014,08-24
187,2015-08-24,Holiday,Local,Ambato,Fundacion de Ambato,False,2015,08-24
271,2016-08-24,Holiday,Local,Ambato,Fundacion de Ambato,False,2016,08-24
327,2017-08-24,Holiday,Local,Ambato,Fundacion de Ambato,False,2017,08-24


In [ ]:
# Check how many stores are affected.
df[df.hol_loc_name == 'Fundacion de Ambato'].store_nbr.unique()

array([23, 50])

## Observation 1:
For kaggle competition purposes: 

We only have to predict for 2017 Aug 16 - Aug 31, only **Aug 24** is a local holiday. 

And it is a **Thursday**. 

Only the city Ambato is affected with stores #23 and #50. 

## Other holiday affects 

### 1. Do all holidays affect sales for all the items in the same way?

Only look into the holidays that are celebrated. Not inlucding the transferred/bridge/workdays 

In [ ]:
# Check holidays:
df_hol_all.transferred.unique()
# The transferred is not right. So for the following code I figured it manually

array([False, 'False', '1', 1.0, 'True'], dtype=object)

In [ ]:
df_hol_all = df[(df.hol_Nat == 1) | (df.hol_Reg == 1) | (df.hol_Loc == 1)  ]
df_hol_true = df_hol_all[(df_hol_all.type_hol_Holiday == 1) & (df_hol_all.transferred.isin([False, 'False']) ) ]  # Only those holidays + celebrated


In [ ]:
# print(df_hol_true)
utility.summary(df_hol_all)

data shape: (241131, 29)


,data type,#missing,%missing,#unique,min,max,first value,second value,third value
date,object,0,0.000000,245,NaN,NaN,2013-01-01,2013-01-01,2013-01-01
year,int64,0,0.000000,5,2013.0,2017.0,2013,2013,2013
month,int64,0,0.000000,12,1.0,12.0,1,1,1
week_number,int64,0,0.000000,42,1.0,53.0,1,1,1
day,int64,0,0.000000,31,1.0,31.0,1,1,1
day_of_week,int64,0,0.000000,7,0.0,6.0,1,1,1
store_nbr,int64,0,0.000000,54,1.0,54.0,25,25,25
type,object,0,0.000000,5,NaN,NaN,D,D,D
cluster,int64,0,0.000000,17,1.0,17.0,1,1,1
city,object,0,0.000000,22,NaN,NaN,Salinas,Salinas,Salinas


In [ ]:
# Calculate the average values for all the items when not holidays or events 
df_non_hol = df[(df.hol_Nat == 0) | (df.hol_Reg == 0) | (df.hol_Loc == 0) ]
df_sale = pd.DataFrame(columns=['family', 'average_sales_non_hol'])
# Loop through each unique family in the filtered dataframe
# Initialize an empty list to collect results
results = []
for item in df_non_hol['family'].unique():
    # Calculate the mean sales for each family and add it to the dataframe
    avg_sales = np.nanmean(df_non_hol[df_non_hol['family'] == item]['sales'])
    # Append a dictionary with 'family' and 'average_sales' to results list
    results.append({'family': item, 'average_sales': avg_sales})

# Convert the results list to a DataFrame
df_sale = pd.DataFrame(results)
# print(df_sale)

In [ ]:
# Add that for holidays 
# National is too complicated. start from Regional
df_sale_regional = df_sale
for hol in df_hol_true.hol_Reg_name.unique():
    if  pd.isna(hol):
        print("Empty holiday name:", hol)
        continue
    # Calculate the average for each item: 
    for item in df_non_hol['family'].unique():
        average_sales = np.nanmean(df_hol_true[(df_hol_true.family == item) | (df_hol_true.hol_Reg_name == hol)]['sales'])
        df_sale_regional.loc[df_sale_regional['family'] == item, hol] = average_sales
#print(df_sale)

Empty holiday name: nan


In [ ]:
df_sale_regional 



,family,average_sales,Provincializacion de Cotopaxi,Provincializacion de Imbabura,Provincializacion de Santo Domingo,Provincializacion Santa Elena
0,AUTOMOTIVE,6.631648,44.627859,21.068690,56.978518,20.505537
1,BABY CARE,0.119957,37.504797,13.301358,49.961819,12.743457
2,BEAUTY,4.037634,41.620107,17.770896,54.000464,17.206168
3,BEVERAGES,2591.841582,2366.274177,2542.396421,2341.124987,2541.159424
4,BOOKS,0.077113,37.504797,13.305035,49.961819,12.743457
5,BREAD/BAKERY,503.754760,480.260280,494.077785,485.330906,493.330401
6,CELEBRATION,9.098565,43.618169,19.956820,55.963250,19.395768
7,CLEANING,1165.626593,1109.070688,1178.806610,1104.860674,1179.009214
8,DAIRY,770.862328,732.259642,767.828669,733.544357,766.918352
9,DELI,288.198944,312.286361,311.849756,320.357213,311.508052


The Comparing doesn't really mean anything especially for some items the average is really low. 

I think for regional average, it's more fair to only consider the regions average for the stores invovled. 

In [ ]:
utility.summary(df_nonhol_for_Hol_region)

In [ ]:
# Re do the average for only the stores invovled. 
# First take out regions
region_list = df[df.hol_Reg == 1].store_nbr.unique()
df_nonhol_for_Hol_region = df[df.store_nbr.isin(region_list)]



# Calculate the average values for all the items when not holidays or events 
df_non_hol = df[(df.hol_Nat == 0) | (df.hol_Reg == 0) | (df.hol_Loc == 0) ]
df_sale = pd.DataFrame(columns=['family', 'average_sales_non_hol'])
# Loop through each unique family in the filtered dataframe
# Initialize an empty list to collect results
results = []
for item in df_non_hol['family'].unique():
    # Calculate the mean sales for each family and add it to the dataframe
    avg_sales = np.nanmean(df_non_hol[df_non_hol['family'] == item]['sales'])
    # Append a dictionary with 'family' and 'average_sales' to results list
    results.append({'family': item, 'average_sales': avg_sales})

# Convert the results list to a DataFrame
df_sale = pd.DataFrame(results)
# print(df_sale)